In [ ]:
# Basic imports
import geopandas as gpd
from pathlib import Path

# Library imports
from geograypher.cameras.derived_cameras import MetashapeCameraSet
from geograypher.meshes import TexturedPhotogrammetryMesh
from geograypher.utils.visualization import show_segmentation_labels
from geograypher.constants import (
    SEGMENTATION_EXAMPLE_LABELS_FILENAME,
    SEGMENTATION_EXAMPLE_MESH_FILENAME,
    SEGMENTATION_EXAMPLE_CAMERAS_FILENAME,
    SEGMENTATION_EXAMPLE_DTM_FILE,
    SEGMENTATION_EXAMPLE_IMAGE_FOLDER,
    SEGMENTATION_EXAMPLE_LABELED_MESH_FILENAME,
    SEGMENTATION_EXAMPLE_RENDERED_LABELS_FOLDER,
    SEGMENTATION_EXAMPLE_LABEL_COLUMN_NAME,
)

# Set constants
You should be able to define most of the behavior from these constants

In [ ]:
## Parameters to control the outputs
# Repeat the labeling process
RETEXTURE = True
# Points less than this height above the DTM are considered ground
# Something is off about the elevation between the mesh and the DTM, this should be a threshold in meters above ground
HEIGHT_ABOVE_GROUND_THRESH = 2
# The image is downsampled to this fraction for accelerated rendering
RENDER_IMAGE_SCALE = 1
# Cameras within this distance of the training data are used in the rendering process
BUFFER_RADIUS_METER = 50
# Tree Radius
TREE_RADIUS_METERS = 2.5
# Downsample target
DOWNSAMPLE_TARGET = 1

## Define the inputs
# The input labels
#"C:\\Users\\david\\dev\\geograypher\\data\\valley_tree_species\\input\\valley_labels.gpkg"
LABELS_FILENAME = SEGMENTATION_EXAMPLE_LABELS_FILENAME
# Render data from this column in the geofile to each image
LABEL_COLUMN_NAME = SEGMENTATION_EXAMPLE_LABEL_COLUMN_NAME
# The mesh exported from Metashape
#"C:\\Users\\david\\dev\\geograypher\\data\\valley_tree_species\\input\\valley_mesh.ply"
MESH_FILENAME = SEGMENTATION_EXAMPLE_MESH_FILENAME
# The camera file exported from Metashape
#"C:\\Users\\david\\dev\\geograypher\\data\\valley_tree_species\\input\\valley_cameras.xml"
CAMERAS_FILENAME = SEGMENTATION_EXAMPLE_CAMERAS_FILENAME
# The digital elevation map exported by Metashape
# "C:\\Users\\david\\dev\\geograypher\\data\\valley_tree_species\\input\\valley_DTM.tif"
DTM_FILE = SEGMENTATION_EXAMPLE_DTM_FILE
# The image folder used to create the Metashape project
#"C:\\Users\\david\\dev\\geograypher\\data\\valley_tree_species\\input\\valley_images"
IMAGE_FOLDER = SEGMENTATION_EXAMPLE_IMAGE_FOLDER

# Where to save the mesh after labeling
#"C:\\Users\\david\\dev\\geograypher\\data\\valley_tree_species\\intermediate\\labeled_mesh.ply"
LABELED_MESH_FILENAME = SEGMENTATION_EXAMPLE_LABELED_MESH_FILENAME
# Where to save the rendering label images
# "C:\\Users\\david\\dev\\geograypher\\data\\valley_tree_species\\intermediate\\rendered_labels"
RENDER_FOLDER = SEGMENTATION_EXAMPLE_RENDERED_LABELS_FOLDER

# Show the geospatial input data

In [ ]:
# Load the data
gdf = gpd.read_file(LABELS_FILENAME)
# Display it, colored by class
gdf.plot(LABEL_COLUMN_NAME, legend=True, vmin=-0.5, vmax=9.5)

# Load a set of cameras and downsample them to the region around annotations

In [ ]:
# Create camera set
camera_set = MetashapeCameraSet(CAMERAS_FILENAME, IMAGE_FOLDER)
# Extract cameras near the training data
training_camera_set = camera_set.get_subset_ROI(
    ROI=LABELS_FILENAME, buffer_radius=BUFFER_RADIUS_METER
)
training_camera_set.vis(force_xvfb=True, frustum_scale=0.5)

# Load the mesh and read texture from geopolygon

In [ ]:
# Create a labeled version of the mesh from the field data
# if not present or requested
if not Path(LABELED_MESH_FILENAME).is_file() or RETEXTURE:
    # Load the downsampled mesh and apply the texture from the vector file
    mesh = TexturedPhotogrammetryMesh(
        MESH_FILENAME,
        downsample_target=DOWNSAMPLE_TARGET,
        ROI=LABELS_FILENAME,
        ROI_buffer_meters=BUFFER_RADIUS_METER,
        texture=LABELS_FILENAME,
        texture_column_name=LABEL_COLUMN_NAME,
        transform_filename=CAMERAS_FILENAME,
    )
    # Get the vertex textures from the mesh
    texture_verts = mesh.get_texture(
        request_vertex_texture=True, try_verts_faces_conversion=False
    )
    mesh.label_ground_class(
        DTM_file=DTM_FILE,
        height_above_ground_threshold=HEIGHT_ABOVE_GROUND_THRESH,
        only_label_existing_labels=True,
        ground_class_name="GROUND",
        set_mesh_texture=True,
    )

    mesh.save_mesh(LABELED_MESH_FILENAME, save_vert_texture=True)
else:
    mesh = TexturedPhotogrammetryMesh(
        LABELED_MESH_FILENAME, transform_filename=CAMERAS_FILENAME
    )

# Show the mesh

In [ ]:
# You can include the camera set, but it's cleaner without it
mesh.vis(camera_set=None, force_xvfb=True)

# Render the labels onto the images

In [ ]:
mesh.save_renders(
    camera_set=training_camera_set,
    render_image_scale=RENDER_IMAGE_SCALE,
    save_native_resolution=True,
    output_folder=RENDER_FOLDER,
)

# Show some of the rendered labels

In [ ]:
show_segmentation_labels(
    label_folder=RENDER_FOLDER,
    image_folder=IMAGE_FOLDER,
    num_show=10,
    label_suffix=".png",
)